In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

C:\Users\shekh\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [49]:
dataFrame = pd.read_csv('C:/Users/shekh/Desktop/rentallisting/extrapolated_trained_data_new_1.csv');
df = pd.DataFrame(dataFrame);
df.columns

Index([u'bathrooms', u'bedrooms', u'building_id', u'created', u'description',
       u'display_address', u'interest_level', u'latitude', u'listing_id',
       u'longitude',
       ...
       u'elevator_laundry', u'incredible_views', u'real_wall_allowed',
       u'post_war', u'high_rise', u'no_fee', u'parking_space',
       u'renovated_apt', u'midrise', u'pet_upon_approval'],
      dtype='object', length=296)

In [3]:
df['interest_level'] = df['interest_level'].astype('category')
df['interest_level_codes'] = df['interest_level'].cat.codes

df['display_address'] = df['display_address'].astype('category')
df['display_address_codes'] = df['display_address'].cat.codes

df['latitude'] = df['latitude'].astype('category')
df['latitude_codes'] = df['latitude'].cat.codes

df['longitude'] = df['longitude'].astype('category')
df['longitude_codes'] = df['longitude'].cat.codes

In [4]:
featuresDF = df.drop('interest_level', axis=1)
featuresDF = featuresDF.drop('building_id', axis=1)
featuresDF = featuresDF.drop('created', axis=1)
featuresDF = featuresDF.drop('description', axis=1)
featuresDF = featuresDF.drop('display_address', axis=1)
featuresDF = featuresDF.drop('latitude', axis=1)
featuresDF = featuresDF.drop('longitude', axis=1)
featuresDF = featuresDF.drop('manager_id', axis=1)
featuresDF = featuresDF.drop('photos', axis=1)
featuresDF = featuresDF.drop('street_address', axis=1)
featuresDF = featuresDF.drop('listing_id', axis=1)

featuresDF.shape

(45342, 289)

In [5]:
#2. Using Variance Threshold for feature selection
selector = VarianceThreshold(threshold=(.9 * (1 - .9)));
temp = selector.fit_transform(featuresDF);

In [6]:
type(temp)

numpy.ndarray

In [7]:
temp.shape

(45342L, 22L)

In [8]:
selected_features = selector.get_support(indices=True)

In [9]:
reducedFeatureDF = featuresDF.iloc[:,selected_features]

In [10]:
reducedFeatureDF.columns

Index([u'bathrooms', u'bedrooms', u'no_of_photos', u'price', u'dish_washer',
       u'fitness_center', u'hardwood_floors', u'dining_room', u'elevator',
       u'doorman', u'dog_allowed', u'roof_deck', u'laundry', u'pre_war',
       u'cat_allowed', u'outdoor_space', u'laundry_bldg', u'no_fee',
       u'interest_level_codes', u'display_address_codes', u'latitude_codes',
       u'longitude_codes'],
      dtype='object')

In [11]:
y_label = reducedFeatureDF['interest_level_codes']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(reducedFeatureDF,y_label, test_size=0.30,random_state=99,stratify=y_label)
# train, validate, test = np.split(reducedFeatureDF.sample(frac=1), [int(.7*len(df)), int(.85*len(df))])
# print len(train);
# print len(validate);
# print len(test);
# X_train = train.drop('interest_level_codes', axis=1)
# X_validate = validate.drop('interest_level_codes', axis=1)
# X_test = test.drop('interest_level_codes', axis=1)

# y_train = train['interest_level_codes']
# y_validate = validate['interest_level_codes']
# y_test = test['interest_level_codes']

print len(X_train)
# print len(X_validate)
print len(X_test)
print len(y_train)
# print len(y_validate)
print len(y_test)

31739
13603
31739
13603


In [13]:
print y_train.value_counts();
# print y_validate.value_counts();
print y_test.value_counts();

1    22551
2     6966
0     2222
Name: interest_level_codes, dtype: int64
1    9665
2    2986
0     952
Name: interest_level_codes, dtype: int64


In [14]:
# def generate_rf(X_train, y_train, X_validate, y_validate):
#     rf = RandomForestClassifier(n_estimators=50, min_samples_leaf=10)
#     rf.fit(X_train, y_train)
#     print "rf score ", rf.score(X_validate, y_validate)
#     return rf

# def combine_rfs(rf_a, rf_b):
#     rf_a.estimators_ += rf_b.estimators_
#     rf_a.n_estimators = len(rf_a.estimators_)
#     return rf_a

In [15]:
# rfs = [generate_rf(X_train, y_train, X_validate, y_validate) for i in xrange(10)]
# rf_combined = reduce(combine_rfs, rfs)

# print "rf combined score", rf_combined.score(X_validate, y_validate)

In [16]:
target_names = ['0', '1', '2'];

In [17]:
# predicted_labels_validation = rf_combined.predict(X_validate)

In [18]:
# print(classification_report(y_validate, predicted_labels_validation, target_names=target_names))

In [19]:
def generate_rf(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators=50, min_samples_leaf=10)
    rf.fit(X_train, y_train)
    print "rf score ", rf.score(X_test, y_test)
    return rf

def combine_rfs(rf_a, rf_b):
    rf_a.estimators_ += rf_b.estimators_
    rf_a.n_estimators = len(rf_a.estimators_)
    return rf_a

In [20]:
rfs = [generate_rf(X_train, y_train, X_test, y_test) for i in xrange(10)]
rf_combined = reduce(combine_rfs, rfs)

print "rf combined score", rf_combined.score(X_test, y_test)

rf score  0.999191354848
rf score  0.999117841653
rf score  0.999117841653
rf score  0.998382709696
rf score  0.999191354848
rf score  0.999117841653
rf score  0.998897302066
rf score  0.999044328457
rf score  0.999117841653
rf score  0.998088656914
rf combined score 0.999411894435


In [21]:
predicted_labels_test = rf_combined.predict(X_test)

In [22]:
print(classification_report(y_test, predicted_labels_test, target_names=target_names))

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       952
          1       1.00      1.00      1.00      9665
          2       1.00      1.00      1.00      2986

avg / total       1.00      1.00      1.00     13603



In [23]:
testDF = pd.read_csv('C:/Users/shekh/Desktop/rentallisting/extrapolated_test_data.csv');

In [24]:
testDF['display_address'] = testDF['display_address'].astype('category')
testDF['display_address_codes'] = testDF['display_address'].cat.codes

testDF['latitude'] = testDF['latitude'].astype('category')
testDF['latitude_codes'] = testDF['latitude'].cat.codes

testDF['longitude'] = testDF['longitude'].astype('category')
testDF['longitude_codes'] = testDF['longitude'].cat.codes

testDF = testDF.drop('Unnamed: 412', axis=1);

In [25]:
testDF.head(5)

,bathrooms,bedrooms,building_id,created,description,display_address,latitude,listing_id,longitude,manager_id,...,semi_floor_through,elevator_laundry,post_war,midrise,mail_room,private_roof_deck,marble_kitchen_great_city_views,display_address_codes,latitude_codes,longitude_codes
0,1.0,1,79780be1514f645d7e6be99a3de696c5,6/11/2016 5:29,Large with awesome terrace--accessible via bed...,Suffolk Street,40.7185,7142618,-73.9865,b1b1852c416d78d7765d746cb1b8921f,...,0,0,0,0,0,0,0,6524,915,419
1,1.0,2,0,6/24/2016 6:36,Prime Soho - between Bleecker and Houston - Ne...,Thompson Street,40.7278,7210040,-74.0000,d0b5648017832b2427eeb9956d966a14,...,0,0,0,0,0,0,0,6576,1008,284
2,1.0,1,3dbbb69fd52e0d25131aa1cd459c87eb,6/3/2016 4:29,New York chic has reached a new level ...,101 East 10th Street,40.7306,7103890,-73.9890,9ca6f3baa475c37a3b3521a394d65467,...,0,0,0,0,0,0,0,412,1036,394
3,2.0,2,6134e7c4dd1a98d9aee36623c9872b49,4/12/2016 5:24,"~Take a stroll in Central Park, enjoy the ente...","Midtown West, 8th Ave",40.7650,6860601,-73.9845,b5eda0eb31b042ce2124fd9e9fcfce2f,...,0,0,0,0,0,0,0,5996,1379,439
4,3.0,3,9d1a51a5df4f0b5650b46da8e5ad6910,4/7/2016 6:14,In a boutique prewar building with 24 hour doo...,Wea,40.7901,6840081,-73.9774,964dc31a872efa33fee9af11f62e843c,...,0,0,0,0,0,0,0,7508,1629,510


In [26]:
featuresTestDF = testDF.drop('building_id', axis=1)
featuresTestDF = featuresTestDF.drop('created', axis=1)
featuresTestDF = featuresTestDF.drop('description', axis=1)
featuresTestDF = featuresTestDF.drop('display_address', axis=1)
featuresTestDF = featuresTestDF.drop('latitude', axis=1)
featuresTestDF = featuresTestDF.drop('longitude', axis=1)
featuresTestDF = featuresTestDF.drop('manager_id', axis=1)
featuresTestDF = featuresTestDF.drop('photos', axis=1)
featuresTestDF = featuresTestDF.drop('street_address', axis=1)
featuresTestDF = featuresTestDF.drop('listing_id', axis=1)

featuresTestDF.shape

(68406, 405)

In [35]:
reducedTestFeatureDF = featuresTestDF.iloc[:,selected_features]

In [37]:
reducedTestFeatureDF.columns

Index([u'bathrooms', u'bedrooms', u'no_of_photos', u'price',
       u'barclays_center', u'balconey', u'corner', u'lofty_layout',
       u'gym_bldg', u'high_ceilings', u'full_time_doorman',
       u'dishwasherhardwood_floors_microwaverenovated', u'green_bldg',
       u'skylight', u'on_site_parking_garage',
       u'ss_appliances_d_w_m_w_recessed_lighting_hardwood_floors_high_ceilings_marble_bath',
       u'residents_lounge', u'dish_washer_two_heating_a_c_units',
       u'three_huge_windows_facing_east_with_unobstructed_views_over_fort_greene',
       u'southern_exposure', u'laundry_room_on_every_other_floor',
       u'backyard'],
      dtype='object')

In [38]:
reducedTestFeatureDF.shape

(68406, 22)

In [40]:
output = rf_combined.predict_proba(reducedTestFeatureDF)

In [41]:
type(output)

numpy.ndarray

In [42]:
output[1]

array([ 0.66866144,  0.29539538,  0.03594318])

In [43]:
rf_combined.classes_

array([0, 1, 2], dtype=int8)

In [46]:
outputDF = pd.DataFrame(output)

In [48]:
outputDF.to_csv('C:/Users/shekh/Desktop/rentallisting/output/random_forest_output_1.csv')